In [ ]:
!nvidia-smi
import feather
import torch
# torch.set_num_threads(32)
import numpy as np
import pickle as pickle
import socket

In [ ]:
from multiprocessing import Pool
from functools import partial
import socket
import feather
import pickle

if socket.gethostname().startswith('chihchung-HP'):
    project_root_dir = '/home/chihchung/GoogleDrive/ppi'
    device = 'cuda'
elif socket.gethostname().startswith('Chihchungs-MacBook-Pro'):
    project_root_dir = '/Volumes/GoogleDrive/My Drive/ppi'
    device = 'cpu'
else:  # colab
    from google.colab import drive
    drive.mount('~/GoogleDrive')
    import sys
    project_root_dir = '/root/GoogleDrive/My Drive/ppi'
    device = 'cuda'
    sys.path.insert(0, '%s/python'%(project_root_dir))
from graph_gradient import * 
from load_python_datasets import load_expr_mat

We are only interested in the secretion of amyloid precursor: APP for now

In [ ]:
G = pickle.load(open('%s/databases/networks/int.db.PCNet.G.p' % project_root_dir, 'rb'))
secMs = feather.read_dataframe(
    '%s/python/data/deep_proteome_median/secM.components.feather' % project_root_dir).iloc[:, 0].to_list()
secPs = feather.read_dataframe(
    '%s/python/data/deep_proteome_median/secP.components.feather' % project_root_dir).iloc[:, 0].to_list()
sec_resident = feather.read_dataframe(
    '%s/python/data/deep_proteome_median/all.secretory.resident.genes.feather' % project_root_dir).iloc[:,
                0].to_list()

APP_pathway_genes = pd.read_csv('%s/databases/2019_AD_GWAS/APP_pathway.csv' % project_root_dir)['Gene'].to_list()
AD_risk_genes = pd.read_csv('%s/databases/2019_AD_GWAS/AD_risk_genes.csv' % project_root_dir)['AD_risk_genes'].to_list()
candidateSecPs = ['APP',# 'MAPT',
                  'ADAM10', #'ADAM10', 'ADAM17', 'ADAM19',
                  'BACE1', #'BACE2',
                  'PSEN1'#,'PSEN2', 'NCSTN', 'APH1A'
                  ]


In [ ]:
# use_expr = 'SC_counts'
# transformation = 'log_sigmoid'

# use_expr = 'SC_tpm'
# transformation = None#'log_sigmoid'

use_expr = 'MSBB_normExp'
transformation = None
# transformation = 'sigmoid'

# use_expr = 'ROSMAP_fpkm'
# transformation = 'log_sigmoid'

expr_mat= load_expr_mat(use_expr=use_expr, project_root_dir = project_root_dir, 
                        transformation=transformation)
patient_names = list(expr_mat.columns)
patient_ids = range(expr_mat.shape[1])
patient_name_dict = dict(zip(patient_ids, patient_names))
allGeneSymbols = list(expr_mat.index)

In [ ]:
calculate_grad = False
summarization = False
calculate_stationary= True
parallel = True
# expr_arr= stats.zscore(np.log(1+expr_arr))
# expr_arr= expit(expr_arr)

def rwr_run(secP):
    try:
        if calculate_grad:
            other_context_genes = list(
                set(APP_pathway_genes + AD_risk_genes) - {secP} - set(secMs) - set(sec_resident)
            a = RwrNode(secp=secP, G=G, secms=secMs, sec_resident=sec_resident,
                        other_context_genes=other_context_genes)
            mask_on = [x for x in a.G.nodes if x not in other_context_genes]
            vt_jacobian_nodes = 'secMs'
        else:
            other_context_genes = []
            a = RwrNode(secp=secP, G=G, secms=secMs, sec_resident=sec_resident,
                        other_context_genes=other_context_genes)
            mask_on = []
            vt_jacobian_nodes = None


    except Exception as e:
        # a = None
        # continuee
        return None

    res_dict = {}
    for patient_id in tqdm(patient_ids):
        res_dict.update({patient_name_dict[patient_id]:
                              a.p_arwr(mask_on=mask_on,
                                      secP_secM_expr=dict(zip(allGeneSymbols, expr_mat.iloc[:, patient_id])),
                                      summarization=summarization,  # 'context_weighted_secM'
                                      vt_jacobian_nodes=vt_jacobian_nodes, calculate_stationary=calculate_stationary
                                      )})

    return res_dict

# import tqdm
if parallel:
    with Pool(4) as p:
        res = list(tqdm(p.imap(rwr_run, candidateSecPs), total=len(candidateSecPs)))
else:
    res = []
    for secP in candidateSecPs:
        res.append(rwr_run(secP))



res_frame = pd.concat([pd.DataFrame.from_dict({(secP, i, j): res_dict[i][j]
                                                for i in res_dict.keys()
                                                for j in res_dict[i].keys()},
                                              orient='index') for secP, res_dict in zip(candidateSecPs, res) if res_dict is not None])
savepath='%s/output/2005_AD_all/%s_%s' %(project_root_dir, use_expr, transformation)
from pathlib import Path
Path(savepath).mkdir(parents=True, exist_ok=True)

res_frame.to_csv('%s/RWR_secPs%s_%s.csv' % (savepath,
                                            'gradient' if calculate_grad else '',
                                            'summarized' if summarization else 'geneLevel'))